# 2025.06.10. (TUE)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Chain & LCEL

### Chain

* Make a **Sequence** of Tasks

* Decompose a complex task multiple simple steps.

#### Concept

* Not only a single LLM CALL, but multiple CALLS or TOOLS RUN in a sequence.

* Reusability & Maintenance

#### Chaining

1. Off-the-shlef Chains(**DEPRECATED!!**)

* preset Chain Class(e.g. `LLMChain`, `SequentialChain`, `SimpleSequentialChain`)

* Initial Architectures.

> NOT RECOMMENDED FOR LANGCHAIN THESE DAYS

2. **LCEL(LangChain Expression Language)**

* Expression-based chaining language

* Concise and declarative syntax. Intuitive and Extensible Chain-structure.

* `Runnable`

* `invoke()`

In [12]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(template = "{item}에 어울리는 이름 {count}개를 만들어 주세요.")
model = ChatOpenAI(model_name = 'gpt-4o-mini')
parser = StrOutputParser()

In [13]:
from langchain import LLMChain

chain = LLMChain(
    prompt = prompt_template,
    llm = model,
    output_parser = parser
)

response = chain.invoke({'item':'가방', 'count':5})

In [17]:
print(response['text'])

물론입니다! 가방에 어울리는 이름 5개를 제안드립니다:

1. **소풍가방** - 일상에서의 소풍 같은 즐거움을 담은 느낌
2. **여행가방** - 모험과 여행을 떠나고 싶은 마음을 담은 이름
3. **스타일백** - 세련된 패션 아이템으로서의 매력을 강조한 이름
4. **마음가방** - 소중한 순간과 추억을 담을 수 있는 따뜻한 느낌의 이름
5. **빛나는 손잡이** - 특별한 순간을 함께하는 가방의 고유한 매력을 나타내는 이름

이 중에서 마음에 드는 이름이 있으면 좋겠습니다!


**Runnable**

* Encapsulation for every tasks

* Constant I/O

* Modulization and Debugging Experience ⬆️

* Methods

    * `invoke()`: a single input

    * `batch()`: multiple input

    * `stream()`: token streaming

    * `ainvoke()`: asynchronous approach

In [ ]:
from langchain_core.runnables import Runnable     # Root class of every Runnables

# Custom Runnable Class Definition
class MyRunnable(Runnable):

    def invoke(self, input_data:str, config:dict=None):
        if config is not None and config.get('lang') == 'en':
            return f"Explain what {input_data} is in a single sentence."
        return f"{input_data}에 대해서 한 문장으로 설명해주세요."

In [23]:
r1 = MyRunnable()
r1.invoke("사과")
r1.invoke("컴퓨터")
r1.invoke("Apple", {"lang":"en"})

'Explain Apple in a single sentence.'

In [ ]:
from langchain_openai import ChatOpenAI

r2 = MyRunnable()
prompt = r2.invoke("MacBook", {"lang":"en"})
prompt = r2.invoke("Langchain")
model = ChatOpenAI(model = 'gpt-4o-mini')
response = model.invoke(prompt)
print(response.content)

The MacBook is a line of portable laptop computers designed and manufactured by Apple, known for their sleek design, high performance, and integration with macOS.


In [32]:
chain = r2 | model
res = chain.invoke('iPhone')
print(res)

content='iPhone은 애플이 개발한 스마트폰으로, 고급스러운 디자인과 사용자 친화적인 인터페이스, 다양한 앱 생태계로 유명합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 18, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bgjh8ZS6NJjg8iWvGWv76NNOetws4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f2cb0c7a-f519-4d15-9489-fa2669085e25-0' usage_metadata={'input_tokens': 18, 'output_tokens': 35, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


**Subclasses of Runnable**

In [48]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

prompt_template = ChatPromptTemplate(
    messages = [
        ('system', '당신은 오랜 경력의 한국 관광 가이드입니다. 여행객들에게 설명하듯이 친절하게 답변해주세요.'),   # Persona(system, user/human, ai/assistant)
        ('human', '{query}')
    ]
)
model = ChatOpenAI(model_name = 'gpt-4o-mini', temperature = 1.0)

guide_chain = prompt_template | model | StrOutputParser()
print(type(guide_chain))    # Runnable: RunnableSequence -> A chain can be a component of another chain.

<class 'langchain_core.runnables.base.RunnableSequence'>


In [49]:
query = '만약 당신이 퇴사하고 나면 직무 경력을 살려 어떤 일에 도전해보고 싶으세요?'
response = guide_chain.invoke({'query':query})

In [50]:
print(response)

안녕하세요! 질문해 주셔서 감사합니다. 만약 제가 퇴사하게 된다면, 그동안 쌓은 관광 가이드로서의 경험을 살려 다양한 관련 분야에 도전해보고 싶습니다.

예를 들어, 문화 콘텐츠 기획이나 여행 관련 교육 프로그램 개발 같은 분야에 관심이 많습니다. 한국의 아름다움과 풍부한 역사, 문화유산을 더 많은 사람들과 나누고 싶기 때문입니다. 또한, 국제 관광업계와의 협업을 통해 한국의 매력을 널리 알리는 프로젝트에도 참여하고 싶습니다.

또한, 디지털 미디어의 발전에 발맞춰 여행 관련 블로그나 유튜브 채널을 운영하는 것도 좋은 아이디어라고 생각합니다. 관광지 소개, 여행 팁, 문화 체험 등을 다양한 형태로 공유하면 더욱 많은 이들과 소통할 수 있을 것 같습니다.

다양한 도전이 기다리겠지만, 저는 언제나 사람들과의 소통과 아름다운 한국의 이야기 전하는 것을 가장 소중하게 여길 것입니다. 감사합니다!


In [53]:
while True:
    query = input("Q: ")
    if query == '!quit':
        break
    res = guide_chain.invoke({'query':query})
    print("User:", query)
    print("AI:", res)
    print("-" * 50)

User: 안녕하세요
AI: 안녕하세요! 한국에 오신 것을 환영합니다. 여행 준비는 잘 되셨나요? 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요. 한국의 아름다운 관광지나 맛있는 음식, 문화에 대해 안내해 드리겠습니다. 즐거운 여행 되세요!
--------------------------------------------------
User: 안녕하세용
AI: 안녕하세요! 반갑습니다. 한국 여행에 대해 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요. 즐거운 여행이 되도록 도와드리겠습니다!
--------------------------------------------------
User: 가성비 좋은 패키지 몇 개 소개해주세요.
AI: 안녕하세요! 가성비 좋은 여행 패키지에 대해 소개해드릴게요. 다양한 옵션들이 있지만, 여기서는 한국을 여행하면서 좋은 경험을 할 수 있는 주요 패키지 몇 가지를 추천드리겠습니다.

1. **서울&부산 2박 3일 패키지**  
   이 패키지는 서울의 유명 관광지와 부산의 해변을 동시에 즐길 수 있는 상품입니다. 서울에서 경복궁, 북촌 한옥마을, 명동을 둘러보고, 부산에서는 해운대와 광안리 해변, 구해수욕장 등을 탐방합니다. 숙소와 식사가 포함되어 있어 이동과 편의성이 좋습니다.

2. **제주도 3일 패키지**  
   제주도를 사랑하는 분들께 추천드리는 패키지입니다. 한라산 국립공원 탐방, 성산 일출봉, 만장굴과 같은 자연경관을 즐길 수 있으며, 제주 전통 음식도 경험할 수 있습니다. 다양한 액티비티(스쿠버다이빙, 투어버스 등)가 포함된 상품도 많이 있으니 확인해보세요.

3. **경주 역사 탐방 패키지**  
   한국의 역사와 문화를 깊게 느낄 수 있는 경주 패키지입니다. 석굴암, 불국사, 경주 국립박물관 등 유네스코 세계문화유산을 탐방합니다. 전통 한정식 저녁이 제공되어 경주의 맛도 함께 경험할 수 있습니다.

4. **강원도 자연 탐방 패키지**  
   대관령, 속초, 양양 등

**RunnableLambda**

In [51]:
from langchain_core.runnables import RunnableLambda

r2 = RunnableLambda(lambda input_data: f"{input_data}를 한 문장으로 설명해줘요.")
r2.invoke("LLM")

'LLM를 한 문장으로 설명해줘요.'

In [52]:
lambda_chain = r2 | model
lambda_chain.invoke("LLM")

AIMessage(content='LLM(대형 언어 모델)은 대량의 텍스트 데이터를 바탕으로 자연어를 이해하고 생성하는 AI 모델입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 19, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BgkAHEWKSfwarHqWg0wBQdzPhAJT7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1348f11-a39c-4995-8412-cc55c0efa6f1-0', usage_metadata={'input_tokens': 19, 'output_tokens': 30, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [54]:
def custom_sum(nums):
    return nums[0] + nums[1]

r2 = RunnableLambda(custom_sum)
r2.invoke({0:10, 1:20})

30

In [ ]:
# cf. invoke(INPUTDATA: str|dict, CONFIGURATIONS:dict)
# INPUTDATA = dict when multiple INPUTs

**RunnablePassthrough**

In [56]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke("안녕하세요")
RunnablePassthrough().invoke({"Key":"Value"})

{'Key': 'Value'}

In [62]:
address_runnable = RunnableLambda(lambda x: "서울시 금천구")
phone_runnable = RunnableLambda(lambda x: '010-1111-2222')

RunnablePassthrough.assign(address = address_runnable, phone = phone_runnable).invoke({'name':'Hong'})

{'name': 'Hong', 'address': '서울시 금천구', 'phone': '010-1111-2222'}

**RunnableSequence**

In [65]:
from langchain_core.runnables import RunnableSequence

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)

chain = run1 | run2
chain.invoke(30)

62

In [67]:
chain2 = RunnableSequence(run1, run2)    # (prompt_template, model, output_parser)
chain2.invoke(30)

62

**RunnableParallel**

In [75]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

run1 = RunnableLambda(lambda x: x + 1)
run2 = RunnableLambda(lambda x: x * 2)
run3 = RunnableLambda(lambda x: x // 3)

runnable = RunnableParallel(
    {
        'result1': run1,
        'result2': run2,
        'result3': run3,
        'result4': RunnablePassthrough()
    }
)

runnable.invoke(20)

{'result1': 21, 'result2': 40, 'result3': 6, 'result4': 20}

#### e.g. LCEL Chain

* To get food name and return recipe of it

* prompt template -> model -> output parser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

class FoodTemplate(Runnable):
    def invoke(self, food, config=None):
        prompt = ""
        if config is not None:
            config = config['configurable'] 
            prompt = "답변을 한국어로 해주세요." if config.get("language") == "kr" else "답변을 일본어로 해주세요." if config.get('language') == "jp" else "답변을 영어로 해주세요."
        print(config, config.get("language"), prompt)
        return f"간단한 가정용 {food} 레시피를 소개해주세요.\n\n"+prompt

prompt_template = FoodTemplate()
model = ChatOpenAI(model = 'gpt-4o-mini', temperature = 1.0)
food_chain = prompt_template | model | StrOutputParser()

In [26]:
# response = food_chain.invoke("Pasta", config={"configurable":{'language':'jp'}})
response = food_chain.invoke("Pasta")
print(response)
# prompt_template.invoke("Pasta", {'language':'jp'})

{} None 답변을 영어로 해주세요.
Sure! Here’s a simple homemade pasta recipe for you.

### Homemade Pasta Recipe

#### Ingredients:
- 2 cups all-purpose flour (plus extra for dusting)
- 3 large eggs
- 1/2 teaspoon salt
- 1 tablespoon olive oil (optional)

#### Instructions:

1. **Mix the Ingredients**: 
   - In a large bowl, combine the flour and salt. Make a well in the center and crack the eggs into the well. If you’re using olive oil, add it now.

2. **Form the Dough**: 
   - Using a fork, beat the eggs gently, then gradually incorporate the flour starting from the edges of the well. Continue until the mixture is too stiff to mix with a fork.

3. **Knead the Dough**: 
   - Turn the dough onto a floured surface and knead it for about 8-10 minutes until it's smooth and elastic. If the dough is sticky, sprinkle a little more flour as needed.

4. **Rest the Dough**: 
   - Wrap the kneaded dough in plastic wrap and let it rest for at least 30 minutes at room temperature. This helps relax the gluten

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent

translate_template = PromptTemplate(
    template = dedent("""
    # Instruction
    주어진 영문 레시피를 번역해주세요
                      
    # Input Data
    1. 레시피: {content}
                      
    2. 번역어: {language}
                      
    # Output
    1. 완성된 문장으로 설명해야 할 경우, -다. -ㅂ니다. 로 작성하지 마세요. **-요** 체로 작성해주세요.
    """)
)

translate_chain = translate_template | model | StrOutputParser() 

translate_chain.invoke({"content":response, "language":"한국어"})

In [84]:
# Instructor's code: English Food Recipe

from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from textwrap import dedent


parser = StrOutputParser()
prompt_template = PromptTemplate(
    template = dedent("""
    # Instruction
    당신은 숙련된 요리 연구가입니다. 요청한 음식의 레시피를 작성해 주세요.
                      
    # Input Data
    음식 이름: {food}
                      
    # Output Indicator: 
    1. 레시피는 영어로 작성해주세요.
    """)
)

model = ChatOpenAI(model = 'gpt-4o-mini', temperature = 1.0)

fchain = prompt_template | model | StrOutputParser()
response = fchain.invoke("Pasta")
print(response)

## Pasta Recipe

### Ingredients:
- 200g dried pasta (spaghetti, penne, or your choice)
- 2 tablespoons olive oil
- 3 cloves garlic, minced
- 1/2 teaspoon red pepper flakes (optional)
- 400g canned diced tomatoes (or fresh tomatoes, diced)
- Salt and pepper to taste
- Fresh basil leaves, chopped (for garnish)
- Grated Parmesan cheese (for serving)

### Instructions:

1. **Cook the Pasta:**
   - In a large pot, bring salted water to a boil. Add the dried pasta and cook according to package instructions until al dente. Reserve 1 cup of pasta water, then drain the pasta and set aside.

2. **Make the Sauce:**
   - In a large skillet, heat the olive oil over medium heat. Add the minced garlic and red pepper flakes (if using), and sauté for about 1 minute until the garlic is fragrant but not browned.
   - Add the canned diced tomatoes to the skillet. Season with salt and pepper. Let the sauce simmer for about 10-15 minutes, stirring occasionally, until it thickens slightly.

3. **Combine Pas

In [102]:
# Instructor's code: Translator
prompt_template_trans = PromptTemplate(
    template = dedent("""
    # Instruction    
    당신은 다국어가 가능한 숙련된 번역가입니다.
    요청된 문장을 {language}로 번역해주세요.
                      
    # Input Data
    {content}

    # Output Indicator
    번역 결과만 출력해주세요.
    반말 하지 마라.           
    """)
)

translate_chain = prompt_template_trans | model | StrOutputParser() 

translate_chain.invoke({"content":response, "language":"한국어"})

'## 파스타 레시피\n\n### 재료:\n- 말린 파스타 200g (스파게티, 페네 또는 선택한 종류)\n- 올리브 오일 2 큰술\n- 다진 마늘 3 쪽\n- 레드 페퍼 플레이크 1/2 작은술 (선택 사항)\n- 깍둑썰기한 캔 토마토 400g (또는 신선한 토마토, 깍둑썰기)\n- 소금과 후추는 취향에 따라\n- 신선한 바질 잎, 다진 것 (장식용)\n- 간 파르메산 치즈 (서빙용)\n\n### 조리법:\n\n1. **파스타 요리:**\n   - 큰 냄비에 소금물 끓이기. 말린 파스타를 넣고 포장 지침에 따라 알 덴테가 될 때까지 요리합니다. 파스타 물 1컵을 남겨두고 파스타를 Drain 여러놓습니다.\n\n2. **소스 만들기:**\n   - 큰 팬에 중불에서 올리브 오일을 가열합니다. 다진 마늘과 레드 페퍼 플레이크(사용하는 경우)를 넣고 마늘의 향이 나지만 갈색이 되지 않도록 약 1분 동안 볶습니다.\n   - 팬에 깍둑썰기한 캔 토마토를 추가합니다. 소금과 후추로 간합니다. 소스가 약간 걸쭉해질 때까지 가끔 저어주며 약 10-15분 동안 끓입니다.\n\n3. **파스타와 소스 결합:**\n   - Drain 한 파스타를 토마토 소스가 있는 팬에 넣습니다. 소스의 원하는 농도가 될 때까지 남겨둔 파스타 물을 조금씩 추가하며 섞습니다.\n\n4. **서빙:**\n   - 파스타를 접시에 담고 다진 신선한 바질로 장식합니다. 간 파르메산 치즈를 위에 뿌려 서빙합니다.\n\n### 즐기세요!\n맛있는 홈메이드 파스타가 준비되었습니다! 추가적인 맛을 위해 좋아하는 채소, 단백질, 또는 허브로 커스터마이즈할 수 있습니다.'

In [71]:
response_kor = translate_chain.invoke({"content":response, "language":"한국어"})
print(response_kor)

## 레시피: 클래식 파스타

### 재료:
- 파스타 400g (스파게티, 펜네 또는 원하는 종류)
- 올리브유 2큰술
- 다진 마늘 3쪽
- 다진 토마토 1캔 (400g) (또는 신선한 토마토 4개, 잘게 썬 것)
- 말린 오레가노 1작은술
- 말린 바질 1작은술
- 소금과 후추는 기호에 맞게
- 파마산 치즈 100g (선택 사항)
- 장식을 위한 신선한 바질 잎 (선택 사항)

### 조리 방법:

1. **파스타 삶기:**
   - 큰 냄비에 소금을 넣고 물을 끓입니다. 파스타를 넣고 포장지에 적힌 대로 알 덴테가 될 때까지 삶아요. 붙지 않게 가끔 저어줘요.
   - 삶은 후, 파스타 물 1컵을 남겨두고, 파스타를 Drain한 후 따로 두어요.

2. **소스 준비하기:**
   - 큰 프라이팬에 중불로 올리브유를 가열해요.
   - 다진 마늘을 넣고 약 1분간 볶아 향이 나도록 하되, 갈색이 나지 않게 조심해요.
   - 다진 토마토(주스 포함), 오레가노, 바질을 팬에 넣어요. 소금과 후추로 간을 해요. 잘 섞어줘요.

3. **끓이기:**
   - 소스를 끓이기 시작하고 10-15분 정도 끓여서 맛이 잘 어우러지도록 해요. 가끔 저어줘요. 소스가 너무 걸쭉해지면 남겨두었던 파스타 물을 조금 추가하여 원하는 농도로 맞춰줘요.

4. **파스타와 소스 결합하기:**
   - 삶은 파스타를 소스가 있는 팬에 넣어요. 파스타가 소스로 고르게 코팅되도록 부드럽게 섞어줘요. 필요하면 남겨둔 파스타 물을 추가해 소스가 파스타에 잘 붙도록 도와줘요.

5. **서빙하기:**
   - 파스타를 접시에 나누어 담고, 원하면 파마산 치즈를 뿌려요. 신선한 바질 잎으로 장식해 주세요.
   - 클래식 파스타 요리를 즐겨요!

### 팁:
- 원하는 대로 채소(시금치, 피망, 애호박 등), 단백질(구운 치킨, 새우, 이탈리안 소시지 등), 또는 매운 맛을 더할 수 있는 향신료를 추가하여 파스타를 개인화할 수 있어요.
- 서빙하기 전에 소스를 맛보고 필요에 따라 간을 조

### Chain + Chain

In [100]:
# food_chain + tranlate_chain
# food -> food_chain
# language -> translate_chain
# food_chain_invoked_result -> {"content":"Our Recipe"} -transmit-> translate_chain

# RunnableParallel
chain = {"content":fchain, "language":RunnablePassthrough(lambda x: x['language'])} | translate_chain

In [92]:
res = chain.invoke({"food":"Pasta", "language":"Japanese"})

In [94]:
print(res)

スパゲッティ・アーリオ・オーリオのレシピ

### 材料:
- 400gのスパゲッティ
- ニンニク6片（薄切り）
- エクストラバージンオリーブオイル1/2カップ
- 赤唐辛子フレーク1/4小さじ（お好みで調整）
- 塩（パスタの茹で水用）
- 刻んだ新鮮なパセリ（飾り用）
- 新鮮にすりおろしたパルメザンチーズ（任意）

### 調理手順:

1. **スパゲッティを茹でる:**
   - 大きな鍋に塩を加えた水を沸騰させる。 
   - スパゲッティを加え、パッケージの指示通りにアルデンテになるまで茹でる。パスタの茹で水を約1カップ取っておき、スパゲッティを水切りして脇に置く。

2. **ガーリックオイルを準備:**
   - 大きなフライパンにオリーブオイルを中火で熱する。
   - フライパンに薄切りのニンニクを加え、黄金色になり香ばしい香りがするまで約2-3分炒める。ニンニクを焦がさないように注意すること、苦味が出ることがある。

3. **赤唐辛子フレークを加える:**
   - ニンニクが黄金色になったら、赤唐辛子フレークを加える。30秒ほどかき混ぜて、オイルに唐辛子の熱を移す。

4. **スパゲッティと合わせる:**
   - 水切りしたスパゲッティをニンニクオイルの入ったフライパンに加える。パスタがオイルとニンニクで均一にコーティングされるまで混ぜる。

5. **濃度を調整:**
   - パスタが乾燥している場合は、取っておいたパスタの茹で水を少しずつ加え、希望の濃度になるまで調整する。

6. **盛り付け:**
   - 火から下ろし、刻んだパセリを加えて混ぜる。
   - 器に盛り付け、さらにパセリとお好みで新鮮にすりおろしたパルメザンチーズで飾ってすぐに提供する。

### ヒント:
- より強い風味を出すために、ニンニクを炒めている間にアンチョビを加えることができます。
- 塩を加える前にパスタの茹で水の味を必ず確認してください。海のように塩辛いはずです。
- この料理は熱々で新鮮なうちにすぐに提供するのが最も美味しいです。

自家製スパゲッティ・アーリオ・オーリオをお楽しみください！


In [96]:
chain_pass = {"content":fchain, "language": RunnableLambda(lambda x: x['language'])} | RunnablePassthrough()
chain_pass.invoke({'food':'Pasta', 'language': 'Japanese'})

{'content': "## Recipe for Pasta\n\n### Ingredients:\n- 2 cups of all-purpose flour\n- 3 large eggs\n- 1/2 teaspoon of salt\n- 1 tablespoon of olive oil (optional)\n- Water (as needed, for dough consistency)\n\n### Instructions:\n\n#### Step 1: Make the Dough\n1. Place the flour on a clean surface or in a large mixing bowl. Make a well in the center.\n2. Crack the eggs into the well. Add the salt and olive oil.\n3. Using a fork, beat the eggs gently, incorporating the flour from the edges of the well until the mixture is combined.\n4. Once the dough starts to come together, use your hands to knead it. If the dough is too dry, add a little water, a teaspoon at a time. If it's too sticky, sprinkle a bit more flour.\n5. Knead the dough for about 8-10 minutes until it is smooth and elastic. Wrap it in plastic wrap and let it rest for at least 30 minutes.\n\n#### Step 2: Roll Out the Dough\n1. Divide the rested dough into four pieces. Keep the pieces you are not using covered to prevent the

In [101]:
chain.invoke({'food':'냉면', 'language':'Japanese'})

'冷麺 (ナengmyeon) レシピ\n\n## 材料：\n\n### 麺のために：\n- 200g 冷麺（そば粉の麺）\n- 水（茹でるため）\n\n### brothのために：\n- 4カップ 牛肉のブロス（または軽いバージョンのために冷水）\n- 大さじ1 醤油\n- 大さじ1 酢（米酢またはりんご酢）\n- 小さじ1 砂糖\n- 塩（味を調えるため）\n- 1-2 束の乾燥昆布（風味を増すためにオプション）\n\n### トッピングのために：\n- 半分に切ったゆで卵1個\n- 千切りきゅうり1/2本\n- 千切り韓国梨1/2個（またはりんごと代用）\n- ゆで牛肉のスライス100g（ブリスケットまたはサーロイン）\n- ごま（トッピング用）\n- やや辛子ペースト（オプション、提供用）\n\n## 説明：\n\n### ステップ1：ブロスを準備する\n1. 鍋に牛肉のブロス（または水）、醤油、酢、砂糖、塩を入れて沸騰させる。\n2. 乾燥昆布を使用する場合は、ブロスに加え、約15分間煮込んで風味を浸透させる。\n3. ブロスを細かいこし器でこして固形物を取り除き、冷蔵庫で冷やす。\n\n### ステップ2：麺を茹でる\n1. 大きな鍋に水を入れて沸騰させる。\n2. 沸騰した水に冷麺を加え、パッケージの指示に従って通常4-6分間茹でる。\n3. 茹で上がったら、麺を水切りし、冷たい流し水で洗って冷やし、余分なデンプンを取り除く。\n\n### ステップ3：料理を盛り付ける\n1. サービングボウルに適量の冷やした麺を置く。\n2. 麺の上に冷たいブロスを注ぎ、完全に浸るが溢れないようにする。\n3. ゆで卵の半分、千切りきゅうり、千切り梨、ゆで牛肉のスライスを麺の上に盛り付ける。\n\n### ステップ4：提供する\n1. ごまを上に振りかけて飾る。\n2. 必要に応じて、辛子ペーストを少し加える。\n3. すぐに提供し、さわやかな冷麺を楽しむ！\n\n## ヒント：\n- 風味を増すために、ブロスを事前に冷凍しておき、それを麺の上に提供して特に冷たく保つことができる。\n- ブロスの調味を自分の味の好みに応じて調整する。'